# Задание 1, обработка данных Доходы и расходы за 2023 год

### импорты

In [1]:
import os
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd
import json
import openpyxl

### Задание 1

In [2]:
def extract_zip(folder_name):
    extracted_folder = f"{folder_name}_extracted"
    os.makedirs(extracted_folder, exist_ok=True)
    for file in os.listdir(folder_name):
        if file.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(folder_name, file), 'r') as zip_ref:
                zip_ref.extractall(extracted_folder)
    return extracted_folder

folder_name = "ФНС_Доходы-Расходы(2023)"
extracted_folder = extract_zip(folder_name)
extracted_folder

'ФНС_Доходы-Расходы(2023)_extracted'

In [7]:
import os
import xml.etree.ElementTree as ET

def process_xml_files(folder):
    data = []
    for root_dir, _, files in os.walk(folder):
        for file in files:
            if file.endswith(".xml"):
                try:
                    tree = ET.parse(os.path.join(root_dir, file))
                    root = tree.getroot()
                    for doc in root.findall(".//Документ"):
                        np = doc.find("СведНП")
                        dr = doc.find("СведДохРасх")
                        if np is not None and dr is not None:
                            inn = np.attrib.get("ИННЮЛ")
                            name = np.attrib.get("НаимОрг")
                            expenses = dr.attrib.get("СумРасход")
                            if inn and name and expenses:
                                try:
                                    expenses_value = float(expenses.replace(',', '.'))
                                    data.append({
                                        "ИНН": inn.strip(),
                                        "НаимОрг": name.strip(),
                                        "СумРасход": expenses_value
                                    })
                                except ValueError:
                                    continue
                except Exception as e:
                    print(f"Ошибка при обработке файла {file}: {e}")
                    continue
    return data

data = process_xml_files(extracted_folder)
data


[{'ИНН': '7100012193',
  'НаимОрг': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФЕЕРИЯДЕНТ"',
  'СумРасход': 17292000.0},
 {'ИНН': '2130228240',
  'НаимОрг': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ДАРИХАУС"',
  'СумРасход': 13945000.0},
 {'ИНН': '9728048501',
  'НаимОрг': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "НКМ СЕРВИС"',
  'СумРасход': 16006000.0},
 {'ИНН': '1328020391',
  'НаимОрг': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФИЗКУЛЬТУРНО-СПОРТИВНЫЙ КЛУБ "ТАЛИНА"',
  'СумРасход': 75589000.0},
 {'ИНН': '7751207788',
  'НаимОрг': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АДОНИС"',
  'СумРасход': 145000.0},
 {'ИНН': '4706043170',
  'НаимОрг': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "СТРОИТЕЛЬНЫЕ РЕШЕНИЯ"',
  'СумРасход': 0.0},
 {'ИНН': '2311325380',
  'НаимОрг': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЛОГИ ТРАНС-23"',
  'СумРасход': 0.0},
 {'ИНН': '7804687523',
  'НаимОрг': 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "СЕВЕР МАРИН"',
  'СумРасход': 16606000.0},
 {'ИНН': '7328

### Задание 2

In [10]:

def save_results(data, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    
    with open(os.path.join(output_folder, "result.json"), "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
   
    df = pd.DataFrame(data)
    max_rows = 1048574
    total_rows = len(df)
    num_sheets = (total_rows // max_rows) + 1
    
    with pd.ExcelWriter(os.path.join(output_folder, "result.xlsx")) as writer:
        for i in range(num_sheets):
            start_row = i * max_rows
            end_row = start_row + max_rows
            df_subset = df.iloc[start_row:end_row]
            sheet_name = f"Sheet{i+1}"
            df_subset.to_excel(writer, sheet_name=sheet_name, index=False)


output_folder = "ФНС_Результаты"
save_results(data, output_folder)


In [11]:
def analyze_max_expenses(data):
    df = pd.DataFrame(data)
    df["Регион"] = df["ИНН"].str[:2]
    results = df.loc[df.groupby("Регион")["СумРасход"].idxmax()]
    return results

max_expenses = analyze_max_expenses(data)
max_expenses

,ИНН,НаимОрг,СумРасход,Регион
1290963,0107029073,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮГ-А...",1.822364e+10,01
357632,0274936578,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЛАЙТ...",2.512535e+11,02
953367,0326562453,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ОЗЕР...",5.053181e+10,03
21341,0411155240,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТОРГ...",1.259651e+10,04
110785,0571012882,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,1.663031e+11,05
...,...,...,...,...
1078227,9308000360,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮЖНЫ...",4.252739e+10,93
866416,9403000070,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮЖНЫ...",7.281537e+10,94
1497276,9500003415,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Б-АГРО""",2.061695e+09,95
421256,9707001911,"""БАНКНОТА"" (ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕН...",1.003623e+12,97


### Задание 3

In [12]:
def analyze_max_expenses(data):
    df = pd.DataFrame(data)
    df["Регион"] = df["ИНН"].str[:2]
    results = df.loc[df.groupby("Регион")["СумРасход"].idxmax()]
    return results

max_expenses = analyze_max_expenses(data)
max_expenses

,ИНН,НаимОрг,СумРасход,Регион
1290963,0107029073,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮГ-А...",1.822364e+10,01
357632,0274936578,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЛАЙТ...",2.512535e+11,02
953367,0326562453,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ОЗЕР...",5.053181e+10,03
21341,0411155240,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТОРГ...",1.259651e+10,04
110785,0571012882,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,1.663031e+11,05
...,...,...,...,...
1078227,9308000360,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮЖНЫ...",4.252739e+10,93
866416,9403000070,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЮЖНЫ...",7.281537e+10,94
1497276,9500003415,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Б-АГРО""",2.061695e+09,95
421256,9707001911,"""БАНКНОТА"" (ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕН...",1.003623e+12,97


#### Вывод максимальных расходов по регионам

In [13]:
for _, row in max_expenses.iterrows():
    print(f"Регион: {row['Регион']}, ИНН: {row['ИНН']}, НаимОрг: {row['НаимОрг']}, СумРасход: {row['СумРасход']}")

Регион: 01, ИНН: 0107029073, НаимОрг: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЮГ-АВТО ЭКСПЕРТ", СумРасход: 18223640000.0
Регион: 02, ИНН: 0274936578, НаимОрг: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЛАЙТ-ИНВЕСТ", СумРасход: 251253454000.0
Регион: 03, ИНН: 0326562453, НаимОрг: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ОЗЕРНОЕ", СумРасход: 50531811000.0
Регион: 04, ИНН: 0411155240, НаимОрг: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ТОРГОВЫЙ ДОМ"ТРАНС-ОЙЛ", СумРасход: 12596512000.0
Регион: 05, ИНН: 0571012882, НаимОрг: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВЛЯЮЩАЯ КОМПАНИЯ "ТОРГОВЫЙ ДВОР ИРЧИ", СумРасход: 166303112000.0
Регион: 06, ИНН: 0601003655, НаимОрг: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "СТРОЙГАЗКОМПЛЕКТ", СумРасход: 12454666000.0
Регион: 07, ИНН: 0701017309, НаимОрг: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЮГАГРОСЕРВИС", СумРасход: 10148040000.0
Регион: 08, ИНН: 0814131130, НаимОрг: ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "СОЛАР", СумРасход: 319303468000.0
Регион

# Задание 2. Сбор дыннх ЕИС "Закупки" 15 баллов

### Задание 1

### Задание 2

### Задание 3

### Задание 4

### Задание 5

### Задание 6